In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
import gzip
import cPickle

In [8]:
# constants
CLASS = 200
SAMPLES_PER_CLASS = 500
TOTAL_SAMPLES = CLASS * SAMPLES_PER_CLASS
COLOR_CHANNELS = 3
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
VAL_SAMPLES = 10000

In [12]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 3])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=200)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=200)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [13]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = cPickle.load(f)
        return loaded_object

In [5]:
tf.logging.set_verbosity(tf.logging.INFO)

# load data
print("loading data from file...")
data = load_zipped_pickle("tinyImageData")
train_data = np.array(data["train"]["data"], dtype = np.float16)
train_labels = np.array(data["train"]["target"], dtype = np.float16)
val_data = np.array(data["val"]["data"], dtype = np.float16)
val_labels = np.array(data["val"]["target"], dtype = np.float16)

loading data from file...


In [14]:
# Create the Estimator
tiny_imagenet_clf = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="tiny_imagenet_cnn_model")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=500)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": train_data}, y=train_labels, batch_size=100,
                                                    num_epochs=None, shuffle=True)
tiny_imagenet_clf.train(input_fn=train_input_fn, steps=20000, hooks=[logging_hook])

# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": val_data}, y=val_labels, num_epochs=1, shuffle=False)
eval_results = tiny_imagenet_clf.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8f591598d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': 1, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'tiny_imagenet_cnn_model', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from tiny_imagenet_cnn_model/model.ckpt-20001
INFO:tensorflow:Saving checkpoints for 20002 into tiny_imagenet_cnn_model/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.00498581  0.00497818  0.00498199 ...,  0.00502014  0.00499725
   0.00499725]
 [ 0.00497818  0.00497437  0.00498581 ...,  0.00501251  0.00500488
   0.00500107]
 [ 0.005

INFO:tensorflow:loss = 3.2266, step = 23502 (1.838 sec)
INFO:tensorflow:global_step/sec: 54.3449
INFO:tensorflow:loss = 3.5859, step = 23602 (1.835 sec)
INFO:tensorflow:global_step/sec: 55.7688
INFO:tensorflow:loss = 3.6738, step = 23702 (1.794 sec)
INFO:tensorflow:global_step/sec: 54.7785
INFO:tensorflow:loss = 3.9844, step = 23802 (1.824 sec)
INFO:tensorflow:global_step/sec: 56.2672
INFO:tensorflow:loss = 4.5781, step = 23902 (1.779 sec)
INFO:tensorflow:global_step/sec: 55.1976
INFO:tensorflow:probabilities = [[ 0.00032282  0.00063705  0.00057507 ...,  0.00054264  0.00032902
   0.00071573]
 [ 0.00086784  0.00105858  0.00071383 ...,  0.00034928  0.00087786
   0.00071669]
 [ 0.00091314  0.00240326  0.00069189 ...,  0.00126266  0.00040364
   0.0012188 ]
 ..., 
 [ 0.00020969  0.0005765   0.00032735 ...,  0.00019085  0.00026298
   0.00051689]
 [ 0.00059175  0.00080824  0.00028706 ...,  0.00041294  0.0006299
   0.00090599]
 [ 0.00015998  0.00023735  0.00021696 ...,  0.00029087  0.00033355


INFO:tensorflow:loss = 3.6484, step = 27502 (1.825 sec)
INFO:tensorflow:global_step/sec: 55.5114
INFO:tensorflow:loss = 3.4609, step = 27602 (1.799 sec)
INFO:tensorflow:global_step/sec: 54.2581
INFO:tensorflow:loss = 3.5742, step = 27702 (1.842 sec)
INFO:tensorflow:global_step/sec: 35.9888
INFO:tensorflow:loss = 3.8691, step = 27802 (2.789 sec)
INFO:tensorflow:global_step/sec: 55.0514
INFO:tensorflow:loss = 3.084, step = 27902 (1.806 sec)
INFO:tensorflow:global_step/sec: 54.9287
INFO:tensorflow:probabilities = [[ 0.00030351  0.00024283  0.00068331 ...,  0.00039268  0.00021863
   0.00011081]
 [ 0.00125122  0.00150299  0.00148582 ...,  0.00297737  0.00110435
   0.00117016]
 [ 0.00021684  0.00008076  0.00011039 ...,  0.00026274  0.00007153
   0.00004804]
 ..., 
 [ 0.00065851  0.00051069  0.00056744 ...,  0.00047064  0.00032091
   0.00010747]
 [ 0.00042939  0.00037026  0.00041628 ...,  0.00031781  0.00012398
   0.00018835]
 [ 0.0001545   0.00007039  0.00012457 ...,  0.00018632  0.00008625


INFO:tensorflow:loss = 3.2246, step = 31502 (1.820 sec)
INFO:tensorflow:global_step/sec: 54.8177
INFO:tensorflow:loss = 2.9336, step = 31602 (1.822 sec)
INFO:tensorflow:global_step/sec: 55.814
INFO:tensorflow:loss = 3.541, step = 31702 (1.792 sec)
INFO:tensorflow:global_step/sec: 55.541
INFO:tensorflow:loss = 3.5684, step = 31802 (1.799 sec)
INFO:tensorflow:global_step/sec: 45.7426
INFO:tensorflow:loss = 2.957, step = 31902 (2.186 sec)
INFO:tensorflow:global_step/sec: 41.7901
INFO:tensorflow:probabilities = [[ 0.0000543   0.00007838  0.00003731 ...,  0.00006914  0.00004905
   0.00010139]
 [ 0.00038671  0.00052071  0.00030136 ...,  0.00054979  0.00074577
   0.00073433]
 [ 0.00049067  0.00030565  0.0006423  ...,  0.00047183  0.00067329
   0.00045919]
 ..., 
 [ 0.00014794  0.00013733  0.00015318 ...,  0.00018549  0.00018919
   0.00015318]
 [ 0.00138664  0.00232315  0.00090981 ...,  0.00325203  0.00164127
   0.00174046]
 [ 0.00006717  0.00006407  0.00003248 ...,  0.00005788  0.00005108
   

INFO:tensorflow:loss = 3.1152, step = 35502 (2.528 sec)
INFO:tensorflow:global_step/sec: 46.6384
INFO:tensorflow:loss = 3.0391, step = 35602 (2.142 sec)
INFO:tensorflow:global_step/sec: 59.0511
INFO:tensorflow:loss = 3.0586, step = 35702 (1.693 sec)
INFO:tensorflow:global_step/sec: 58.6541
INFO:tensorflow:loss = 3.0664, step = 35802 (1.706 sec)
INFO:tensorflow:global_step/sec: 33.9139
INFO:tensorflow:loss = 2.6602, step = 35902 (2.955 sec)
INFO:tensorflow:global_step/sec: 28.1157
INFO:tensorflow:probabilities = [[ 0.00000089  0.00000137  0.00000244 ...,  0.00000703  0.00000179
   0.00000364]
 [ 0.00054598  0.00046158  0.00055456 ...,  0.00063133  0.00056124
   0.00168228]
 [ 0.00011659  0.00018203  0.00024962 ...,  0.00028419  0.00016308
   0.0002327 ]
 ..., 
 [ 0.00002289  0.00002807  0.0000264  ...,  0.00012243  0.00003809
   0.00004148]
 [ 0.00004417  0.00002676  0.00002223 ...,  0.00005758  0.00004631
   0.00007337]
 [ 0.0001421   0.00010437  0.00009137 ...,  0.00021923  0.00024939

INFO:tensorflow:loss = 3.3496, step = 39502 (2.595 sec)
INFO:tensorflow:global_step/sec: 56.4345
INFO:tensorflow:loss = 3.2949, step = 39602 (1.768 sec)
INFO:tensorflow:global_step/sec: 54.8013
INFO:tensorflow:loss = 2.7656, step = 39702 (1.824 sec)
INFO:tensorflow:global_step/sec: 56.289
INFO:tensorflow:loss = 3.3164, step = 39802 (1.777 sec)
INFO:tensorflow:global_step/sec: 55.2214
INFO:tensorflow:loss = 3.8594, step = 39902 (1.810 sec)
INFO:tensorflow:Saving checkpoints for 40001 into tiny_imagenet_cnn_model/model.ckpt.
INFO:tensorflow:Loss for final step: 3.1152.
INFO:tensorflow:Starting evaluation at 2017-12-10-23:02:48
INFO:tensorflow:Restoring parameters from tiny_imagenet_cnn_model/model.ckpt-40001
INFO:tensorflow:Finished evaluation at 2017-12-10-23:02:49
INFO:tensorflow:Saving dict for global step 40001: accuracy = 0.005, global_step = 40001, loss = 7.68033
{'loss': 7.6803303, 'global_step': 40001, 'accuracy': 0.0049999999}


In [ ]:
print(data)